 ## Techniques to reduce runtime in Pandas

### Key Takeaways
#### 1. Use Vectorisation 
#### 2. When converting a column to datetime, specify the format
#### 3. The built in function .iterrows is quicker than for loops
#### 4. Use .eval expressions on very large datasets
#### 5. When combining multiple dataframes, use pd.concat. But only use it once, on a list of all the dataframes    
        5i. If the original data is in dictionary form, it is much quicker to create the combined dataframe as the final step on a list of dictionaries

In [122]:
#Import modules
import pandas as pd
import numpy as np
rng = np.random.RandomState(123)
#Import data
date_data = pd.read_csv('data/transactions.csv')
time_data = pd.read_csv('data/temperature_and_wind.csv',index_col=0)


### 1. Vectorization is much quicker than .apply function
    Vectorisation is the ability to run an operation across a whole Series, Index, or even DataFrame at once

In [74]:
df = pd.DataFrame(np.random.randint(0, 11, size=(1000000, 5)), columns=('a','b','c','d','e'))

In [77]:
%%timeit
def func(a,b,c,d,e):
    if e == 10:
        return c*d
    elif (e < 10) and (e>=5):
        return c+d
    elif e < 5:
        return a+b
df['new'] = df.apply(lambda x: func(x['a'], x['b'], x['c'], x['d'], x['e']), axis=1)

19.5 s ± 997 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [78]:
%%timeit
df['new'] = df['c'] * df['d'] #default case e = =10
mask = df['e'] < 10
df.loc[mask,'new'] = df['c'] + df['d']
mask = df['e'] < 5
df.loc[mask,'new'] = df['a'] + df['b']

97.2 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 2. Specifying the format of datetime when converting

    Pandas will take a significant amount of runtime to interpret the data without formatting provided. Note this is the case for Datetime conversion, date conversion is not affected

##### DateTime

In [53]:
df = time_data[:10000].copy()

@timeit(repeat=3, number=10)
def convert(df, column_name):
    return pd.to_datetime(df[column_name])
    
df['Date Time'] = convert(df, 'Date Time')

Best of 3 trials with 10 function calls per trial:
Function `convert` ran in average of 0.644 seconds.



In [54]:
df = time_data[:10000].copy()

@timeit(repeat=3, number=10)
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],format='%d.%m.%Y %H:%M:%S')

df['Date Time'] = convert_with_format(df, 'Date Time')

Best of 3 trials with 10 function calls per trial:
Function `convert_with_format` ran in average of 0.021 seconds.



##### Date

In [64]:
df = date_data.copy()

@timeit(repeat=3, number=10)
def convert(df, column_name):
    return pd.to_datetime(df[column_name])
    
df['date'] = convert(df, 'date')

Best of 3 trials with 10 function calls per trial:
Function `convert` ran in average of 0.378 seconds.



In [73]:
df = date_data.copy()

@timeit(repeat=3, number=10)
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],format='%Y-%m-%d')

df['date'] = convert_with_format(df, 'date')

Best of 3 trials with 10 function calls per trial:
Function `convert_with_format` ran in average of 0.378 seconds.



### 3. Use .iterrows() or .apply() instead of looping through range

In [113]:
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],format='%d.%m.%Y %H:%M:%S')

def times_of_day(hour,temp):
    if hour < 6:
        rate = 3
    elif hour < 12:
        rate = 2
    elif hour < 16:
        rate = 1
    elif hour <20:
        rate = 2
    else:
        rate = 3
    return rate*temp


##### For Loop

In [162]:
df = time_data[:100000].copy()


@timeit(repeat=3, number=10)
def get_temp_perhour(df):
    df['Date Time'] = convert_with_format(df, 'Date Time')
    temp_hour_list = []
    for i in range(len(df)):                        ##### Loopoing through range
        hour = df.iloc[i]['Date Time'].hour
        temp = df.iloc[i]['T (degC)']
        temp_hour_list.append(times_of_day(hour,temp))

    df['temp-hour'] = temp_hour_list
get_temp_perhour(df)

Best of 3 trials with 10 function calls per trial:
Function `get_temp_perhour` ran in average of 11.981 seconds.



#### iterrows

In [163]:
df = time_data[:100000].copy()

def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],format='%d.%m.%Y %H:%M:%S')

@timeit(repeat=3, number=10)
def get_temp_perhour(df):
    df['Date Time'] = convert_with_format(df, 'Date Time')
    temp_hour_list = []
    for index, row in df.iterrows():                ###### Using iterrows
        hour = row['Date Time'].hour
        temp = row['T (degC)']
        temp_hour_list.append(times_of_day(hour,temp))
    df['temp-hour'] = temp_hour_list

get_temp_perhour(df)

Best of 3 trials with 10 function calls per trial:
Function `get_temp_perhour` ran in average of 2.459 seconds.



#### The apply function is quicker than both of the above

In [164]:
df = time_data[:100000].copy()

def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],format='%d.%m.%Y %H:%M:%S')

@timeit(repeat=3, number=10)
def apply_get_temphour(df):
    df['Date Time'] = convert_with_format(df, 'Date Time')
    df['temp-hour'] = df.apply(                                            #Using apply function
        lambda row: times_of_day(row['Date Time'].hour,row['T (degC)']),
        axis=1
    )
apply_get_temphour(df)

Best of 3 trials with 10 function calls per trial:
Function `apply_get_temphour` ran in average of 0.839 seconds.



#### Vectorisation is still quicker than all of the above
##### It won't use the original function above but vectorisation can be coded in multiple ways

    Here .isin() is used though mask can be also used like in case 1

In [165]:
df = time_data[:100000].copy()

@timeit(repeat=3, number=10)
def use_isin(df):
    df['Date Time'] = convert_with_format(df, 'Date Time')

    time = df['Date Time'].dt.hour
    early = time.isin(range(0,6))
    morning = time.isin(range(6,12))
    afternoon = time.isin(range(12,16))
    evening = time.isin(range(16,20))
    late = time.isin(range(20,25))


    df.loc[early, 'temp-hour'] = df.loc[early, 'T (degC)'] * 3
    df.loc[morning, 'temp-hour'] = df.loc[morning, 'T (degC)'] * 2
    df.loc[afternoon, 'temp-hour'] = df.loc[afternoon, 'T (degC)'] * 1
    df.loc[evening, 'temp-hour'] = df.loc[evening, 'T (degC)'] * 2
    df.loc[late, 'temp-hour'] = df.loc[late, 'T (degC)'] * 3

use_isin(df)

Best of 3 trials with 10 function calls per trial:
Function `use_isin` ran in average of 0.016 seconds.



    The built in pd.cut can be used instead of .isin for less lines of code and slight runtime improvement

In [166]:
df = time_data[:100000].copy()

@timeit(repeat=3, number=10)
def use_cut(df):
    df['Date Time'] = convert_with_format(df, 'Date Time')

    hour_factor = pd.cut(x=df['Date Time'].dt.hour,
                        bins=[0,5,11,15,19,24],
                        include_lowest=True,
                        labels = [3,2,1,4,5]).astype(int)
    
    hour_factor = hour_factor.replace({4:2,5:3})
    
    df['temp-hour'] =  hour_factor * df['T (degC)']

use_cut(df)

Best of 3 trials with 10 function calls per trial:
Function `use_cut` ran in average of 0.011 seconds.



    Numpy's digitize function works in the same way as pd.cut, but it's slightly quicker

In [167]:
df = time_data[:100000].copy()

@timeit(repeat=3, number=10)
def use_digitize(df):
    df['Date Time'] = convert_with_format(df, 'Date Time')

    bins = np.digitize(df['Date Time'].dt.hour.values,
                        bins=[6,12,16,20,24])
    factor = np.array([3,2,1,2,3])
    
    df['temp-hour'] =  factor[bins]

use_digitize(df)

Best of 3 trials with 10 function calls per trial:
Function `use_digitize` ran in average of 0.008 seconds.



### 4. Use eval() for large dataframes

    Why it is quicker for large dataframes: https://jakevdp.github.io/PythonDataScienceHandbook/03.12-performance-eval-and-query.html

##### Eval uses string expressions to efficiently compute operations. In the case below it's over 50% faster

In [181]:
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4,df5 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(5))

In [182]:
%timeit (-df1 * df2 / (df3 + df4) - df5)

190 ms ± 6.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [183]:
%timeit pd.eval('-df1 * df2 / (df3 + df4) - df5')

57.9 ms ± 2.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 5. What is the fastest way to concat multiple dataframes?

In [109]:
def generate_df(i):
    df = pd.DataFrame(i,index=[0],columns=['x', 'y'])
    return df

def generate_dictionary(i):
    return {'x':i,'y':i}

#### Concat each dataframe in a for loop (Slowest)

In [115]:
@timeit
def concat_each_df(): 
    combined = pd.DataFrame()

    for i in range(1,1000): # demo only
        df = generate_df(i) # df is created here
        combined = pd.concat([combined, df])
    return combined.reset_index()

In [116]:
concat_each_df()

Best of 3 trials with 1000 function calls per trial:
Function `concat_each_df` ran in average of 0.244 seconds.



,index,x,y
0,0,1,1
1,0,2,2
2,0,3,3
3,0,4,4
4,0,5,5
...,...,...,...
994,0,995,995
995,0,996,996
996,0,997,997
997,0,998,998


#### Use concat only once on a list of all dataframes

In [117]:
@timeit
def list_of_dfs(): 
    dfs = []
    for i in range(1,1000): # demo only
        df = generate_df(i) # df is created here
        dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)

In [118]:
list_of_dfs()

Best of 3 trials with 1000 function calls per trial:
Function `list_of_dfs` ran in average of 0.141 seconds.



,x,y
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
994,995,995
995,996,996
996,997,997
997,998,998


#### Keep data in dictionary format. Create a final dataframe from list of dictionaries

In [119]:
@timeit
def df_from_dict():
    rows = []
    for i in range(1, 1000):
    # Instead of generating a dataframe, generate a dictionary
        dictionary = generate_dictionary(i)
        rows.append(dictionary)

    return pd.DataFrame(rows)

In [120]:
df_from_dict()

Best of 3 trials with 1000 function calls per trial:
Function `df_from_dict` ran in average of 0.001 seconds.



,x,y
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
994,995,995
995,996,996
996,997,997
997,998,998
